In [1]:
import duckdb
import random
import pandas as pd

In [2]:
PATH = '../data_2025/fulldata_07_24_04_P_일반음식점.csv'
df = pd.read_csv(PATH, encoding='cp949')
df = df.copy()

C:\Users\PCN\AppData\Local\Temp\ipykernel_23452\2085175914.py:2: DtypeWarning: Columns (11,17,39,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH, encoding='cp949')


In [3]:
# PATH = '../data/fulldata_07_24_04_P_일반음식점.csv'
# df = pd.read_csv(PATH, encoding='cp949')
# df = df.copy()

In [4]:
def split_address(address):
    parts = address.split()
    
    if len(parts) >= 4:
        city_name = parts[0]  # 예: 전라북도
        district = parts[1]  # 예: 남원시
        town = parts[2]  # 예: 대강면
        road_name = ' '.join(parts[3:-1])  # 예: 대강월산길 (주소의 마지막 요소 전까지 모두 도로명으로 처리)
        bungee = parts[-1]  # 예: 37-16 (주소의 마지막 요소는 항상 번지로 처리)
        
        return city_name, district, town, road_name, bungee
    else:
        return None, None, None, None, None

In [5]:
def prepro_df(df):
    random_value = random.randint(10120, 10200)
    df = df[:random_value].loc[:, ['개방서비스명', '인허가일자', '폐업일자','영업상태명', '도로명전체주소', '사업장명', '업태구분명', '남성종사자수', '여성종사자수']]
    df['도로명전체주소'] = df['도로명전체주소'].fillna('')
    df['폐업일자'] = df['폐업일자'].fillna('운영중')
    df.loc[:, '도로명전체주소'] = [address.split(',')[0] for address in df['도로명전체주소']]

    df[['시도', '시군구', '읍면동', '도로명', '번지']] = df['도로명전체주소'].apply(lambda x: pd.Series(split_address(x)))
    df.loc[:, 'num'] = 1
    df = df.rename(columns={'도로명전체주소': '소재지', '사업장명': '시설명', '업태구분명': '구분'})
    df = df.reset_index().drop(columns='index')
    return df
    # df.to_csv(path, index=False)

In [6]:
sorted_day_df = df.sort_values(by='인허가일자',ascending=False)
sorted_day_df['인허가일자'] = pd.to_datetime(sorted_day_df['인허가일자'], format='%Y-%m-%d', errors='coerce')
sorted_day_df['인허가일자'].fillna(method='ffill', inplace=True)

df_2025 = sorted_day_df[sorted_day_df['인허가일자'].dt.year.astype(int) == 2025].reset_index().drop('index', axis=1)
df_2024 = sorted_day_df[sorted_day_df['인허가일자'].dt.year.astype(int) == 2024].reset_index().drop('index', axis=1)
df_2023 = sorted_day_df[sorted_day_df['인허가일자'].dt.year.astype(int) == 2023].reset_index().drop('index', axis=1)
df_2022 = sorted_day_df[sorted_day_df['인허가일자'].dt.year.astype(int) == 2022].reset_index().drop('index', axis=1)
df_2021 = sorted_day_df[sorted_day_df['인허가일자'].dt.year.astype(int) == 2021].reset_index().drop('index', axis=1)
df_2020 = sorted_day_df[sorted_day_df['인허가일자'].dt.year.astype(int) == 2020].reset_index().drop('index', axis=1)

C:\Users\PCN\AppData\Local\Temp\ipykernel_23452\366942668.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sorted_day_df['인허가일자'].fillna(method='ffill', inplace=True)
C:\Users\PCN\AppData\Local\Temp\ipykernel_23452\366942668.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sorted_day_df['인허가일자'].fillna(method='ffill', inplace=True)


In [7]:
sorted_day_df.head()

,번호,개방서비스명,개방서비스아이디,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,...,공장생산직직원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지,Unnamed: 47
1776513,1776514,일반음식점,07_24_04_P,4261000,4261000-101-2025-00064,2025-08-29,NaN,1,영업/정상,1,...,0.0,NaN,0.0,0.0,N,50.00,NaN,NaN,NaN,NaN
1776472,1776473,일반음식점,07_24_04_P,3170000,3170000-101-2025-00216,2025-08-29,NaN,1,영업/정상,1,...,0.0,NaN,0.0,0.0,N,52.32,NaN,NaN,NaN,NaN
1776446,1776447,일반음식점,07_24_04_P,6510000,6510000-101-2025-00581,2025-08-29,NaN,1,영업/정상,1,...,0.0,NaN,0.0,0.0,N,22.10,NaN,NaN,NaN,NaN
1776447,1776448,일반음식점,07_24_04_P,6510000,6510000-101-2025-00580,2025-08-29,NaN,1,영업/정상,1,...,0.0,NaN,0.0,0.0,N,333.33,NaN,NaN,NaN,NaN
1776040,1776041,일반음식점,07_24_04_P,3320000,3320000-101-2025-00083,2025-08-29,NaN,1,영업/정상,1,...,0.0,NaN,0.0,0.0,N,96.54,NaN,NaN,NaN,NaN


In [8]:
df_2025 = prepro_df(df_2025)
df_2024 = prepro_df(df_2024)
df_2023 = prepro_df(df_2023)
df_2022 = prepro_df(df_2022)
df_2021 = prepro_df(df_2021)
df_2020 = prepro_df(df_2020)

In [9]:
print(len(df_2020))
print(len(df_2021))
print(len(df_2022))
print(len(df_2023))
print(len(df_2024))
print(len(df_2025))

10151
10151
10158
10157
10184
10146


In [10]:
df_2025[:3]

,개방서비스명,인허가일자,폐업일자,영업상태명,소재지,시설명,구분,남성종사자수,여성종사자수,시도,시군구,읍면동,도로명,번지,num
0,일반음식점,2025-08-29,운영중,영업/정상,,제3회 공근 소맥축제,한식,0.0,0.0,None,None,None,None,None,1
1,일반음식점,2025-08-29,운영중,영업/정상,서울특별시 금천구 두산로5길 58,돌리고,한식,0.0,0.0,서울특별시,금천구,두산로5길,,58,1
2,일반음식점,2025-08-29,운영중,영업/정상,제주특별자치도 제주시 서사로 101,박한별무도학원,분식,0.0,0.0,제주특별자치도,제주시,서사로,,101,1


In [11]:
df_2024[:3]

,개방서비스명,인허가일자,폐업일자,영업상태명,소재지,시설명,구분,남성종사자수,여성종사자수,시도,시군구,읍면동,도로명,번지,num
0,일반음식점,2024-12-31,2025-03-24,폐업,서울특별시 강남구 테헤란로52길 21,삼겹장인,한식,0.0,0.0,서울특별시,강남구,테헤란로52길,,21,1
1,일반음식점,2024-12-31,운영중,영업/정상,서울특별시 종로구 돈화문로5가길 31,러브버드케이지 (LOVE BIRD CAGE),기타,0.0,0.0,서울특별시,종로구,돈화문로5가길,,31,1
2,일반음식점,2024-12-31,2025-01-31,폐업,경기도 수원시 권선구 세화로 134,솔트팩토리,기타,0.0,0.0,경기도,수원시,권선구,세화로,134,1


In [12]:
df_2023[:3]

,개방서비스명,인허가일자,폐업일자,영업상태명,소재지,시설명,구분,남성종사자수,여성종사자수,시도,시군구,읍면동,도로명,번지,num
0,일반음식점,2023-12-29,운영중,영업/정상,전라남도 화순군 화순읍 광덕로 169-3,돼지전설,식육(숯불구이),0.0,0.0,전라남도,화순군,화순읍,광덕로,169-3,1
1,일반음식점,2023-12-29,운영중,영업/정상,경상남도 창원시 의창구 동읍 용잠로35번길 1,째리네점빵,한식,0.0,0.0,경상남도,창원시,의창구,동읍 용잠로35번길,1,1
2,일반음식점,2023-12-29,운영중,영업/정상,경상남도 창원시 마산합포구 남성로 120,살아있는 꽃게찜,한식,0.0,0.0,경상남도,창원시,마산합포구,남성로,120,1


In [13]:
df_2022[:3]

,개방서비스명,인허가일자,폐업일자,영업상태명,소재지,시설명,구분,남성종사자수,여성종사자수,시도,시군구,읍면동,도로명,번지,num
0,일반음식점,2022-12-30,운영중,영업/정상,강원특별자치도 삼척시 척주로 17,황제갈비,한식,0.0,0.0,강원특별자치도,삼척시,척주로,,17,1
1,일반음식점,2022-12-30,운영중,영업/정상,인천광역시 중구 용유서로 294-3,을왕리 고기카페,식육(숯불구이),0.0,0.0,인천광역시,중구,용유서로,,294-3,1
2,일반음식점,2022-12-30,2023-01-02,폐업,강원특별자치도 삼척시 새천년도로 100 (정하동),정라응급처치전문의용소방대,한식,0.0,0.0,강원특별자치도,삼척시,새천년도로,100,(정하동),1


In [14]:
df_2021[:3]

,개방서비스명,인허가일자,폐업일자,영업상태명,소재지,시설명,구분,남성종사자수,여성종사자수,시도,시군구,읍면동,도로명,번지,num
0,일반음식점,2021-12-31,운영중,영업/정상,인천광역시 서구 청마로112번길 20,인생주막,한식,0.0,0.0,인천광역시,서구,청마로112번길,,20,1
1,일반음식점,2021-12-31,운영중,영업/정상,충청남도 당진시 석문면 해명4로3길 30,황금성,중국식,0.0,0.0,충청남도,당진시,석문면,해명4로3길,30,1
2,일반음식점,2021-12-31,2023-07-17,폐업,전라남도 여수시 중앙로 43,아구의왕,기타,0.0,0.0,전라남도,여수시,중앙로,,43,1


In [15]:
df_2020[:3]

,개방서비스명,인허가일자,폐업일자,영업상태명,소재지,시설명,구분,남성종사자수,여성종사자수,시도,시군구,읍면동,도로명,번지,num
0,일반음식점,2020-12-31,운영중,영업/정상,경상북도 구미시 임은1길 3,황금육꼬기,한식,NaN,NaN,경상북도,구미시,임은1길,,3,1
1,일반음식점,2020-12-31,운영중,영업/정상,충청북도 청주시 흥덕구 오송읍 만수1길 11,푸드박스 오송점,김밥(도시락),NaN,NaN,충청북도,청주시,흥덕구,오송읍 만수1길,11,1
2,일반음식점,2020-12-31,2021-05-31,폐업,경기도 안산시 상록구 광덕1로 365,씨티오브드림,기타,NaN,NaN,경기도,안산시,상록구,광덕1로,365,1


In [16]:
df_2020.to_csv('../data_2025/restaurant_2020.csv', index=False)
df_2021.to_csv('../data_2025/restaurant_2021.csv', index=False)
df_2022.to_csv('../data_2025/restaurant_2022.csv', index=False)
df_2023.to_csv('../data_2025/restaurant_2023.csv', index=False)
df_2024.to_csv('../data_2025/restaurant_2024.csv', index=False)
df_2025.to_csv('../data_2025/restaurant_2025.csv', index=False)

In [17]:
conn = duckdb.connect('../data_2025/database.db') # database 연결

In [18]:
# 데이터 삭제
conn.execute('DROP TABLE IF EXISTS restaurant_2025')
conn.execute('DROP TABLE IF EXISTS restaurant_2024')
conn.execute('DROP TABLE IF EXISTS restaurant_2023')
conn.execute('DROP TABLE IF EXISTS restaurant_2022')
conn.execute('DROP TABLE IF EXISTS restaurant_2021')
conn.execute('DROP TABLE IF EXISTS restaurant_2020')

In [19]:
# 데이터 저장
conn.execute('CREATE TABLE restaurant_2025 AS SELECT * FROM df_2025')
conn.execute('CREATE TABLE restaurant_2024 AS SELECT * FROM df_2024')
conn.execute('CREATE TABLE restaurant_2023 AS SELECT * FROM df_2023')
conn.execute('CREATE TABLE restaurant_2022 AS SELECT * FROM df_2022')
conn.execute('CREATE TABLE restaurant_2021 AS SELECT * FROM df_2021')
conn.execute('CREATE TABLE restaurant_2020 AS SELECT * FROM df_2020')

In [20]:
# database close
conn.close()